# Step by step reconstruction of the model

In [18]:
import numpy as np
import tensorflow as tf
import os
import cv2

In [19]:
def load_data(img_dir, ending):
    return np.array([cv2.imread(os.path.join(img_dir, img)) for img in os.listdir(img_dir) if img.endswith(ending)])


In [38]:
X = load_data("3Shapes2_large/", "im1.png")
Y = load_data("3Shapes2_large/","im2.png")

In [41]:
X.shape

(10, 128, 128, 3)

In [40]:
X = X[:10]
Y = Y[:10]

In [42]:
def create_graph(img1, img2):
    img1_64 = tf.nn.max_pool(img1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    img2_64 = tf.nn.max_pool(img2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    #Motion encoder

    #First convolution: 5x5x96
    weights = tf.Variable(tf.random_normal([5,5,3,3]))
    bias = tf.Variable(tf.zeros([3,]))
    logits = tf.nn.conv2d(img1_64, filter=weights, strides=[1,1,1,1], padding='SAME')
    logits = tf.add(logits, bias)
    logits = tf.nn.relu(logits)
    
    return img1_64, img2_64, logits


In [43]:
def train(prediction, y):
    l2_loss = tf.reduce_mean(tf.square(prediction - y))
    optimizer = tf.train.AdamOptimizer(0.01)
    train = optimizer.minimize(l2_loss)
    
    return l2_loss, train

In [60]:
# run the model
def run(X, Y, n_epochs = 10, batch_size = 10):
    img1 = tf.placeholder(shape=(batch_size,128,128,3), dtype=tf.float32, name="s1s")
    img2 = tf.placeholder(shape=(batch_size,128,128,3), dtype=tf.float32)
    
    img1_64, img2_64, output = create_graph(img1, img2)
    loss, training = train(output, img2_64)
    
    with tf.Session() as sess:
        for epoch in range(n_epochs):
            
            sess.run(tf.global_variables_initializer())
            
            print('Epoch %i/%i' % (epoch+1, n_epochs))
            cumulative_loss = 0.0
            
            for batch_num in range(int(X.shape[0]/batch_size)-1):
                # get x and y
                x = X[batch_size*batch_num: batch_size*batch_num + batch_size]
                y = Y[batch_size*batch_num: batch_size*batch_num + batch_size]
                
                # run train and loss
                _, batch_loss = sess.run([training,loss], feed_dict={img1:x, img2:y})
                print("\t\tbatch_loss:", batch_loss)
                cumulative_loss += batch_loss
            
            print("\tEpoch's loss:", cumulative_loss)
                

# TODO
1. add the next convolution (working with only one resized img (64x64))

In [59]:
run(X, Y, 10, 2)

Epoch 1/10
		batch_loss: 82363.7
		batch_loss: 56363.9
		batch_loss: 59719.2
		batch_loss: 54523.5
	Epoch's loss: 252970.332031
Epoch 2/10
		batch_loss: 1.45739e+06
		batch_loss: 1.18637e+06
		batch_loss: 901312.0
		batch_loss: 916315.0
	Epoch's loss: 4461387.75
Epoch 3/10
		batch_loss: 58004.5
		batch_loss: 50856.4
		batch_loss: 51355.9
		batch_loss: 46259.2
	Epoch's loss: 206475.96875
Epoch 4/10
		batch_loss: 2.22307e+06
		batch_loss: 1.94724e+06
		batch_loss: 1.38992e+06
		batch_loss: 969053.0
	Epoch's loss: 6529277.4375
Epoch 5/10
		batch_loss: 1.27121e+06
		batch_loss: 1.0492e+06
		batch_loss: 791415.0
		batch_loss: 528558.0
	Epoch's loss: 3640381.125
Epoch 6/10
		batch_loss: 47402.6
		batch_loss: 44134.1
		batch_loss: 46456.3
		batch_loss: 41619.9
	Epoch's loss: 179612.839844
Epoch 7/10
		batch_loss: 2.45434e+06
		batch_loss: 2.06022e+06
		batch_loss: 1.6945e+06
		batch_loss: 1.89592e+06
	Epoch's loss: 8104973.25
Epoch 8/10
		batch_loss: 496721.0
		batch_loss: 331395.0
		batch_lo